In [148]:
import pandas as pd
import numpy as np

In [149]:
crowd_df = pd.read_csv("../../data/crowd_data.tsv", sep="\t")
crowd_df.head()

,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,7QT,Is this triple correct or incorrect?,$0.50,2,2133U7HKDLO,Submitted,40,40%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,yes,yes
2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
4,1,7QT,Is this triple correct or incorrect?,$0.50,5,2134U7HKDMM,Submitted,2,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN


In [150]:
def convert_percentage_to_float(percentage_string):
    """

    Convert a percentage string to a float.

    For example, '99%' becomes 0.99.

    """

    if isinstance(percentage_string, str):
        return float(percentage_string.strip("%")) / 100

    return percentage_string


crowd_df["LifetimeApprovalRate"] = crowd_df["LifetimeApprovalRate"].apply(
    convert_percentage_to_float
)

In [151]:
crowd_df.describe()

,HITId,AssignmentId,WorkTimeInSeconds,LifetimeApprovalRate,AnswerID
count,305.000000,305.000000,305.000000,305.000000,304.000000
mean,31.000000,153.000000,108.800000,0.747443,1.473684
std,17.635752,88.190136,95.548706,0.204038,0.500130
min,1.000000,1.000000,2.000000,0.400000,1.000000
25%,16.000000,77.000000,4.000000,0.690000,1.000000
50%,31.000000,153.000000,120.000000,0.800000,1.000000
75%,46.000000,229.000000,200.000000,0.980000,2.000000
max,61.000000,305.000000,337.000000,0.990000,2.000000


In [152]:
crowd_df = crowd_df.drop(crowd_df[crowd_df.LifetimeApprovalRate < 0.5].index)
crowd_df = crowd_df.drop(crowd_df[crowd_df.WorkTimeInSeconds < 40].index)

In [153]:
crowd_df.describe()

,HITId,AssignmentId,WorkTimeInSeconds,LifetimeApprovalRate,AnswerID
count,183.000000,183.000000,183.000000,183.000000,183.000000
mean,31.000000,152.251366,174.065574,0.881257,1.508197
std,17.655121,88.228115,66.240206,0.100444,0.501304
min,1.000000,1.000000,50.000000,0.700000,1.000000
25%,16.000000,76.500000,120.000000,0.800000,1.000000
50%,31.000000,152.000000,200.000000,0.850000,2.000000
75%,46.000000,227.500000,240.000000,0.980000,2.000000
max,61.000000,303.000000,337.000000,0.990000,2.000000


In [154]:
crowd_df.head()

,HITId,HITTypeId,Title,Reward,AssignmentId,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
0,1,7QT,Is this triple correct or incorrect?,$0.50,1,2133ICYWE97,Submitted,60,0.99,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
2,1,7QT,Is this triple correct or incorrect?,$0.50,3,928UJANWZ12,Submitted,50,0.98,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,4,1726JMZQW,Submitted,80,0.70,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
6,2,7QT,Is this triple correct or incorrect?,$0.50,7,2133ICYWE97,Submitted,120,0.99,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN
7,2,7QT,Is this triple correct or incorrect?,$0.50,8,928UJANWZ12,Submitted,60,0.98,wd:Q603545,wdt:P2142,4300000,1.0,CORRECT,NaN,NaN


In [155]:
# Filter the dataset for rows where the answer is 'INCORRECT' and there is a provided fix
incorrect_with_fix = crowd_df[(crowd_df['AnswerLabel'] == 'INCORRECT') & (crowd_df['FixPosition'].notna()) & (crowd_df['FixValue'].notna())]

# Extract the necessary columns
incorrect_with_fix = incorrect_with_fix[['HITId', 'Input1ID', 'Input2ID', 'Input3ID', 'FixPosition', 'FixValue']]

# Display a sample of these rows
incorrect_with_fix.head()


,HITId,Input1ID,Input2ID,Input3ID,FixPosition,FixValue
10,3,wd:Q16911843,wdt:P577,2014-01-18,Object,2014-02-18
28,6,wd:Q48313910,wdt:P577,2018-02-24,Object,2019-02-24
36,8,wd:Q217010,wdt:P2142,698491347,Object,698491348
51,11,wd:Q1339195,ddis:indirectSubclassOf,wd:Q27096213,Object,wd:Q72
60,13,wd:Q61928601,ddis:indirectSubclassOf,wd:Q95074,Object,wd:Q94074


In [156]:
# Function to apply the correction based on FixPosition and FixValue
def apply_correction(row):
    if row['FixPosition'].lower() == 'subject':
        row['Input1ID'] = row['FixValue']
    elif row['FixPosition'].lower() == 'predicate':
        row['Input2ID'] = row['FixValue']
    elif row['FixPosition'].lower() == 'object':
        row['Input3ID'] = row['FixValue']
    return row

# Applying the corrections
corrected_data = incorrect_with_fix.apply(apply_correction, axis=1)

# Display a sample of the corrected data
corrected_data[['HITId', 'Input1ID', 'Input2ID', 'Input3ID']].head()


,HITId,Input1ID,Input2ID,Input3ID
10,3,wd:Q16911843,wdt:P577,2014-02-18
28,6,wd:Q48313910,wdt:P577,2019-02-24
36,8,wd:Q217010,wdt:P2142,698491348
51,11,wd:Q1339195,ddis:indirectSubclassOf,wd:Q72
60,13,wd:Q61928601,ddis:indirectSubclassOf,wd:Q94074


In [142]:
# Recalculate the majority vote
majority_vote = crowd_df.groupby("HITId")["AnswerLabel"].agg(
    lambda x: x.value_counts().index[0]
)

final_answers = majority_vote.reset_index()

# Get the distribution of answers for each HITId
answer_distribution = (
    crowd_df.groupby("HITId")["AnswerLabel"].value_counts().unstack(fill_value=0)
)

# Join the majority vote table with the answer distribution
final_answers_with_distribution = final_answers.join(answer_distribution, on="HITId")

# Include the correct inputs
correct_inputs = (
    crowd_df[crowd_df["AnswerLabel"] == "CORRECT"]
    .groupby("HITId")
    .first()[["Input1ID", "Input2ID", "Input3ID"]]
)

final_answers_with_distribution = final_answers_with_distribution.join(
    correct_inputs, on="HITId"
)

# Adding the batch identifier (HITTypeId) to the final table
final_answers_with_batch = final_answers_with_distribution.join(
    crowd_df[["HITId", "HITTypeId"]].drop_duplicates().set_index("HITId"), on="HITId"
)

# Display the updated table with HITTypeId
final_answers_with_batch.head()

,HITId,AnswerLabel,CORRECT,INCORRECT,Input1ID,Input2ID,Input3ID,HITTypeId
0,1,CORRECT,2,1,wd:Q11621,wdt:P2142,792910554,7QT
1,2,CORRECT,3,0,wd:Q603545,wdt:P2142,4300000,7QT
2,3,CORRECT,2,1,wd:Q16911843,wdt:P577,2014-01-18,7QT
3,4,INCORRECT,0,3,NaN,NaN,NaN,7QT
4,5,CORRECT,3,0,wd:Q1628022,wdt:P577,1951-01-01,7QT


In [143]:
from statsmodels.stats.inter_rater import fleiss_kappa

# Grouping data by HITTypeId and then by HITId within each group, and counting the answers
grouped_data = crowd_df.groupby(['HITTypeId', 'HITId', 'AnswerLabel']).size().unstack(fill_value=0)

# Calculating Fleiss' Kappa for each HITTypeId
fleiss_kappa_results = {}
for hit_type_id, group in grouped_data.groupby(level=0):
    contingency_table = group.reset_index(level=0, drop=True)  # Drop the HITTypeId index
    kappa = fleiss_kappa(contingency_table)
    fleiss_kappa_results[hit_type_id] = kappa

# Converting results to a DataFrame for better presentation
fleiss_kappa_df = pd.DataFrame(list(fleiss_kappa_results.items()), columns=['HITTypeId', 'FleissKappa'])

fleiss_kappa_df.head()

,HITTypeId,FleissKappa
0,7QT,0.236364
1,8QT,0.040000
2,9QT,0.199110


In [144]:
final_joined_df = final_answers_with_batch.merge(fleiss_kappa_df, on='HITTypeId').reset_index(drop=True)
final_joined_df.head()


,HITId,AnswerLabel,CORRECT,INCORRECT,Input1ID,Input2ID,Input3ID,HITTypeId,FleissKappa
0,1,CORRECT,2,1,wd:Q11621,wdt:P2142,792910554,7QT,0.236364
1,2,CORRECT,3,0,wd:Q603545,wdt:P2142,4300000,7QT,0.236364
2,3,CORRECT,2,1,wd:Q16911843,wdt:P577,2014-01-18,7QT,0.236364
3,4,INCORRECT,0,3,NaN,NaN,NaN,7QT,0.236364
4,5,CORRECT,3,0,wd:Q1628022,wdt:P577,1951-01-01,7QT,0.236364


In [145]:
final_only_correct = final_joined_df.drop(final_joined_df[final_joined_df.AnswerLabel == "INCORRECT"].index)
final_only_correct.head()

,HITId,AnswerLabel,CORRECT,INCORRECT,Input1ID,Input2ID,Input3ID,HITTypeId,FleissKappa
0,1,CORRECT,2,1,wd:Q11621,wdt:P2142,792910554,7QT,0.236364
1,2,CORRECT,3,0,wd:Q603545,wdt:P2142,4300000,7QT,0.236364
2,3,CORRECT,2,1,wd:Q16911843,wdt:P577,2014-01-18,7QT,0.236364
4,5,CORRECT,3,0,wd:Q1628022,wdt:P577,1951-01-01,7QT,0.236364
6,7,CORRECT,2,1,wd:Q171300,wdt:P2142,267000000,7QT,0.236364


In [159]:
final_only_correct.to_csv("crowdsourcing_final.csv", index=False)